# Les centres d'intérêts des députés influencent-ils leur participation aux séances en commission permanente ?

## Introduction

Introduction ici: contexte général



présenter la motivation

<b> Stratégie d'idenfication </b>
 présenter la stratégie d'idenfication
 
 <b> Algorithmes </b>
 
 Présenter l'aspect plus algorithmique 
 
 
A demander : 

Récupérer le champ lexical


préférences des députés 



jouer sur le nombre de catégories, voir le threshold de déclenchement 

idée: 

• définir des catégories
•gradient de préférences pour le député
•gradient de thématiques pour la séance
•voir les paires de thématiques qui reviennent le plus souvent les unes à la suite des autres pour en déduire une proximité entre les différentes thématiques 
    - idée probabilité d'avoir un mot à la suite d'un autre 
    
• calculer une distance thématique/préférence, voir le pouvoir prédictif de cette distance


première étape pour les mots: créer une boucle, 1 ligne numero du député, une autre avec toutes ses interventions à la suite (sans traitement) 
filtrer pour les séances en hémicycle uniquement

à partir de ça voir les mots qui sortent le plus souvent pour chacun des députés




 


In [1]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

## Créer des catégories thématiques

Première chose : créer les catégories thématiques (regrouper les champs lexicaux). Agreger toutes les interventions de tous les députés / prendre uniquement les séances en hémicycle et agréger pour tous les députés

A la fin, avoir des mots qui permettent de classifier une thématique de séance
faire du train/test pour voir si ça marche bien


La première chose à faire est de récupérer un fichier contenant toutes les interventions de tous les députés. On importe le fichier <TT>words_mps.txt</TT> qui contient l'ensemble des interventions en hémicycle de l'ensemble des députés de la XIVème législature. Plus précisément, la colonne 0 correspond à l'id du député dans notre base de données (qui nous sera utile plus tard pour retrouver la présence des députés en commission ainsi que leur groupe parlementaire). La colonne 1 est la concaténation brute de toutes ses interventions que l'on va ensuite nettoyer pour extraire les centres d'intérêts de chacun des députés.

Nous utilisons un script autonome pour extraire les données à partir d'une base SQL (cf. annexes)

<b>INférer les catégories à partir des séances ?  + extraire le tire (voir les trigrams/4grammes)</b>

In [356]:
import pandas 
import numpy as np

#Attention à indiquer l'emplacement local du fichier words_mps.txt (accessible en local uniquement)

df=pandas.read_csv('/Users/Gabriel/Desktop/words_mps2.txt', sep='\t',header=None)
#df=pandas.read_csv('/Users/Hugo/Documents/Cours/ENSAE/Python/python_deputes/words_mps.txt', sep='\t',header=None,na_filter=False)
df1 = df.values
df2 = np.array(df)
df.head()

,0,1
0,1,"('<p>Bla-bla-bla !</p>',)(""<p>Vous n'avez enco..."
1,2,"(""<p>Ce n'est pas fini !</p>"",)('<p>Lamentable..."
2,3,"(""<p>J'ai bien entendu la position du Gouverne..."
3,4,"(""<p>C'est vous qui le faites !</p>"",)(""<p>Mon..."
4,5,"(""<p>Monsieur le président, monsieur le minist..."


In [358]:
#On va transformer le tableau des prises de parole des députés en hémicycle en liste unidimensionnelle
#où les items 0...n correspondent aux prises de parole du premier député, n+1, ... n+k à celles du deuxième etc. 
#on nettoye au passage les chaines

document=[]
for i in range(0,648):
    z = str(df2[i,1])
    interventions=[]
    z = z.replace('"<p>',"")
    z = z.replace("('<p>","")
    z = z.replace("',)(","")
    z = z.replace('",)(','')
    z = z.replace('",)','')
    z = z.replace("',)",'')
    z = z.replace("<p>",'')
    interventions=z.split("</p>")
    document.extend(interventions)

print(len(document))

817602


On procède ensuite à la vectorisation et au k-means pour extraire des clusters

In [361]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import nltk

minidoc=document[:1000]

stpwrd = stopwords.words('french')
stpwrd.append('les')
vectorizer = TfidfVectorizer(stop_words=stpwrd)
X = vectorizer.fit_transform(minidoc)

#NOMBRE DE CLUSTERS: arbitraire au début
true_k = 10
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

#On a les clusters 

print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k-5):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :20]:
        print(' %s' % terms[ind]),
    print


Top terms per cluster:
Cluster 0:
 majorité
 ils
 faites
 gouvernement
 pourquoi
 compris
 plus
 ministre
 loi
 entre
 bien
 moins
 gauche
 8230
 projet
 france
 leurs
 hollande
 donc
 parti
Cluster 1:
 être
 banques
 autres
 loi
 activités
 plus
 monde
 bancaire
 régulation
 bancaires
 faire
 ans
 crise
 peut
 tout
 français
 matière
 peu
 comme
 pense
Cluster 2:
 amendement
 8211
 monsieur
 cette
 rapporteur
 retiré
 ministre
 adopté
 donc
 général
 cet
 bien
 fait
 8230
 ni
 plus
 question
 également
 rapport
 commission
Cluster 3:
 cela
 non
 plus
 jamais
 logements
 paris
 totalement
 sociaux
 logement
 bien
 quoi
 tout
 monsieur
 très
 rien
 cet
 aussi
 fait
 amendements
 bancs
Cluster 4:
 madame
 rapporteure
 ministre
 présidente
 fait
 effectivement
 comme
 mme
 franchement
 collègues
 amendements
 amendement
 revenu
 contraire
 groupe
 bancs
 si
 monsieur
 france
 lepetit


In [4]:
print(type(order_centroid))

NameError: name 'order_centroid' is not defined

problème: sélection du nombre de clusters, on va devoir appliquer différentes méthodes et choisir le bon nombre
    de clusters
    
Can, F.; Ozkarahan, E. A. (1990). "Concepts and effectiveness of the cover-coefficient-based clustering methodology for text databases". ACM Transactions on Database Systems. 15 (4): 483. doi:10.1145/99935.99938. especially see Section 2.7.    

http://www.xavierdupre.fr/app/mlstatpy/helpsphinx/c_clus/kmeans.html

Choisir le nombre de clusters optimaux

## Créer un profil de préférence à chaque député

On a un espace de dimension k, l'idée va être d'attribuer un nuplet de coordonnées au député dans cet espace. ça nous permet de le situer dans l'espace des thématiques 

tout va sommer à 1, donc voir pour enlever un cluster éventuellement (si c'est bien fait, le cluster des salamaleks)

side: grâce aux informations sur le député, on peut avoir la localisation en fonction de l'âge, de l'obédience politique, du sexe, de l'expérience, etc

ces coordonnées définissent le profil de préférence du député

## Attribuer une thématique aux séances en commission

Rebelotte mais pour les séances en commission, très similaire aux députés (cf. script autonome)

In [2]:
import pandas

#Attention à indiquer l'emplacement local du fichier words_meetings.txt (accessible en local uniquement)

df=pandas.read_csv('/Users/Gabriel/Desktop/words_meetings.txt', sep='\t',header=None)
#df=pandas.read_csv('/Users/Hugo/Documents/Cours/ENSAE/Python/python_deputes/words_meetings.txt', sep='\t',header=None)
df.head()
df3 = df.values
df4 = np.array(df)
df.head()

,0,1
0,3,"(""<p>Commission du développement durable et de..."
1,4,"(""<p>COMMISSION DES AFFAIRES CULTURELLES ET DE..."
2,5,"(""<p>La Commission des affaires économiques s'..."
3,6,"(""<p>La Commission a procédé, sous la présiden..."
4,7,"(""<p>COMMISSION DES AFFAIRES SOCIALES</p><p>Je..."


Le problème ici est d'inférer à partir des mots qui ont été prononcés des thématiques en fonction des clusters identifiés pour les députés. Les séances en hémicycle étant pour l'essentiel le prolongement des séances en commission, les clusters devraient être appropriés (restriction aux séances en hémicycle "législatives" pour supprimer les questions au gouvernement qui n'ont pas grand chose à voir) 

Pour chaque liste, créer un gradient de proximité avec les n clusters identifiés 
chaque intervention est classée dans un des clusters

on fait le ratio et celui qui arrive en premier devient la thématique de la séance


a la fin, on a un vecteur kx2 où k est le nombre de séances et pour chaque séance on a l'id et le thème (numéro du cluster)

## Voir comment la proximité thématique prédit la participation

### Utiliser les données de présence

On a une matrice qui a $ \sum_{i=1}^n m_i$ lignes où $m_i$ désigne le nombre de séances auxquelles aurait du assister le député $i$ compte tenu de sa commission assignée et de l'intervalle de son mandat (au moins rien, au plus une législature). La première colonne correspond au numéro du député (répété $m$ fois), la seconde au numéro de la séance (unique) et la troisième indique si le député était présent (1) ou non (0).

L'idée est dans un premier temps de calculer pour tous les couples (députés,séance) la distance entre le thème de la séance et les préférences du député

Ensuite une fois qu'on aura la distance l'inclure en variable indépendante et voir son pouvoir prédictif sur la présence/participation en commission. 

Problèmes: interprétation de la distance ? 

### Estimation du modèle

Le modèle s'écrit comme suit : 
$$
pres_{it}=\beta_0+\beta_1 dist_{it}+\beta_X\textbf{X}+\varepsilon_{it}
$$

Où pres=0,1 indique si le député était présent ou non et dist est la distance (euclidienne pour commencer) entre le profil thématique de la séance et le profil de préférences du député et X un set de contrôles

voir où mettre le prédictif là dedans, voir s'il faut le mettre éventuellement

## Bibliographie


<span>&#8226;</span> Benjamin Monnery & Maxime Le Bihan, 2018. "Can Public and Private Sanctions Discipline Politicians? Evidence from the French Parliament," EconomiX Working Papers 2018-21, University of Paris Nanterre, EconomiX.

<span>&#8226;</span> Wu, A. H. (2017). Gender stereotyping in academia: Evidence from economics job market rumors forum.
